In [1]:
import os
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
@dataclass
class SegyPosFile:
    name: str
    path: str
    fin_traceno: int = 0
    datetime: list = field(default_factory=list)
    traceno: list = field(default_factory=list)
    cdp_x: list = field(default_factory=list)
    cdp_x_smoothed: list = field(default_factory=list)
    cdp_y_smoothed: list = field(default_factory=list)
    cdp_x_cartesian_smoothed: list = field(default_factory=list)
    cdp_y_cartesian_smoothed: list = field(default_factory=list)
    cdp_y: list = field(default_factory=list)
    year: list = field(default_factory=list)
    day: list = field(default_factory=list)
    hour: list = field(default_factory=list)
    minute: list = field(default_factory=list)
    second: list = field(default_factory=list)

In [3]:
def read_segypos(pos_files, finedict, baddict, posobj_list, year, utm_coords=False):
    for pos_file in pos_files:
        segy_name = os.path.splitext(os.path.basename(pos_file))[0]
        pos_obj = SegyPosFile(name=segy_name, path=pos_file)
        
        number = 0
        has_error = False
        was_before = False
        
        with open(pos_file, 'r') as file1:
            file_content = file1.read().splitlines()
            
            for line in file_content[1:]:
                line_content = line.split()

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm34n, always_xy=True)

In [3]:
from pyproj import Proj, CRS, Transformer

crs_wgs84 = CRS.from_epsg(4326)
crs_utm35n = CRS.from_epsg(32635)
crs_utm34n = CRS.from_epsg(32634)

transformer = Transformer.from_crs(crs_wgs84, crs_utm34n, always_xy=True)

data_path = r'H:\SBP_DATA\ABP53'
# track
save_to = r'H:\SBP_DATA\abp53_sbp_profile_names.txt'

is_utm = False
year = 2023
pos_files = fes.file_ext_search('.txt', data_path, recursive=True)
print(len(pos_files))

segy_pos_objs = []
bad_data_dict = {}
fine_data_dict = {}

read_segypos(pos_files, fine_data_dict, bad_data_dict, segy_pos_objs, year, utm_coords = is_utm)
process_track(segy_pos_objs, transformer, window_length=201, smooth=True, utm_coords=is_utm)
# save_track(segy_pos_objs, save_to)
save_track_to_radex(segy_pos_objs)

Searching *.txt files in directory:H:\SBP_DATA\ABP53
1421
Number of bad lines in 144-31_W2_SLF2305211039_LL_car_rawpos: 248
Number of bad lines in 144-31_W2_SLF2305211049_LL_car_rawpos: 189
Number of bad lines in 144-31_W2_SLF2305211051_LL_car_rawpos: 3522
Number of bad lines in 166-39_W2_SLF2305212046_LL_car_rawpos: 1762
Number of bad lines in Limacina-0_W2_SLF2305091535_LL_env_rawpos: 37
Can not smooth file abp53-1_W2_SLF2305120601_LL_car_rawpos
Can not smooth file g.172-177_W2_SLF2305221051_LL_env_rawpos
Can not smooth file g_23-13_W2_SLF2305251021_LL_car_rawpos
Can not smooth file g_206-207_станция-3_W2_SLF2305240903_LL_car_rawpos
Can not smooth file g_207_станция-4_W2_SLF2305241017_LL_car_rawpos
Can not smooth file Limacina-0_W2_SLF2305081642_LL_env_rawpos
Can not smooth file Limacina-0_W2_SLF2305111109_LL_car_rawpos
File 144-31_W2_SLF2305210252_LL_car_rawpos is done 1 of 1421
File 144-31_W2_SLF2305210302_LL_car_rawpos is done 2 of 1421
File 144-31_W2_SLF2305210312_LL_car_rawpos i